In [3]:
import os
import pandas as pd
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Tokenizer
from tqdm import tqdm_notebook

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def tokenizer_gptj():
    pass
def tokenizer_gpt2_large():
    pass
def tokenizer_gpt_neo_2_7():
    pass
def tokenizer_opt_6_7b():
    pass
def tokenizer_bloomz():
    pass
def tokenizer_bloom():
    pass
def tokenizer_bloom_7b():
    pass
def tokenizer_opt_125m():
    pass
def tokenizer_bloom_560m():
    pass
def model_gptj():
    pass
def model_gpt2_large():
    pass
def model_gpt_neo_2_7():
    pass
def model_opt_6_7b():
    pass
def model_bloomz():
    pass
def model_bloom():
    pass
def model_bloom_7b():
    pass
def model_opt_125m():
    pass
def model_bloom_560m():
    pass

### Loading the model(s)

1. Uncomment the three strings (tokenizer, model, model.to(device)) for the model(s) that you would like to test.

In [2]:
# tokenizer_gptj = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", cache_dir=None)
# model_gptj = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", cache_dir=None)
# model_gptj.to(device)
# tokenizer_gpt2_large = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=None)
# model_gpt2_large = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir=None)
# model_gpt2_large.to(device)
# tokenizer_gpt_neo_2_7 = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-2.7B', cache_dir=None)
# model_gpt_neo_2_7 = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-2.7B', cache_dir=None)
# model_gpt_neo_2_7.to(device)
# tokenizer_opt_6_7b = AutoTokenizer.from_pretrained("facebook/opt-6.7b", use_fast=False, cache_dir=None)
# model_opt_6_7b = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b", torch_dtype=torch.float16, cache_dir=None)
# model_opt_6_7b.to(device)
# tokenizer_bloomz = AutoTokenizer.from_pretrained('bigscience/bloomz-3b', cache_dir=None)
# model_bloomz = AutoModelForCausalLM.from_pretrained('bigscience/bloomz-3b', cache_dir=None)
# model_bloomz.to(device)
# tokenizer_bloom = AutoTokenizer.from_pretrained('bigscience/bloom-3b', cache_dir=None)
# model_bloom = AutoModelForCausalLM.from_pretrained('bigscience/bloom-3b', cache_dir=None)
# model_bloom.to(device)
# tokenizer_opt_125m = AutoTokenizer.from_pretrained("facebook/opt-125m", use_fast=False, cache_dir=None)
# model_opt_125m = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", torch_dtype=torch.float16, cache_dir=None)
# model_opt_125m.to(device)
# tokenizer_bloom_560m = AutoTokenizer.from_pretrained('bigscience/bloom-560m', cache_dir=None)
# model_bloom_560m = AutoModelForCausalLM.from_pretrained('bigscience/bloom-560m', cache_dir=None)
# model_bloom_560m.to(device)
# tokenizer_bloom_7b = AutoTokenizer.from_pretrained('bigscience/bloom-7b1', cache_dir=None)
# model_bloom_7b = AutoModelForCausalLM.from_pretrained('bigscience/bloom-7b1', cache_dir=None)
# model_bloom_7b.to(device)

### Making causal models generate responses, acting as chatbots 

In [16]:
dict_models = {"model": {
        "gptj": model_gptj,
        "gpt2_large": model_gpt2_large,
        "gpt_neo_2_7": model_gpt_neo_2_7,
        "opt_6_7b": model_opt_6_7b,
        "bloomz": model_bloomz,
        "bloom": model_bloom,
        "bloom_7b": model_bloom_7b,
        "bloom_560m": model_bloom_560m,
        "opt_125m": model_opt_125m,
    },
    "tokenizer": {
        "gptj": tokenizer_gptj,
        "gpt2_large": tokenizer_gpt2_large,
        "gpt_neo_2_7": tokenizer_gpt_neo_2_7,
        "opt_6_7b": tokenizer_opt_6_7b,
        "bloomz": tokenizer_bloomz,
        "bloom": tokenizer_bloom,
        "bloom_7b": tokenizer_bloom_7b,
        "bloom_560m": tokenizer_bloom_560m,
        "opt_125m": tokenizer_opt_125m,
    }
}

In [17]:
def generate_text(text, model_name='gptj', len_gen_text=40):
    input_ids = dict_models['tokenizer'][model_name](f"{text}", return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = dict_models['model'][model_name].generate(input_ids, max_length=len(dict_models['tokenizer'][model_name](text)['input_ids'])+len_gen_text, 
    min_length=8, top_p=0.9, temperature=0.9,do_sample=True, pad_token_id=dict_models['tokenizer'][model_name].eos_token_id)
    output = dict_models['tokenizer'][model_name].decode(outputs[0], skip_special_tokens=True)
    return output

In [21]:
def ask(question, chat_log=None, model='gptj'):
    prompt = f'{chat_log}Human: {question}\nAI:'
    full_response = generate_text(prompt, model_name=model)
    gpt_response = full_response.replace(chat_log, "")
    sub_str = "\nHuman:"
    re = gpt_response.split(sub_str)
    res=re[0]
    return res
    
def get_tokens_num(utterance, model='gptj'):
  input_ids = dict_models["tokenizer"][model].encode(str(utterance), return_tensors="pt").cuda()
  tokens = list(map(dict_models["tokenizer"][model].decode, input_ids[0]))
  tokens_num = len(tokens)
  return tokens_num


def create_chat_log_item(user, utterance, model='gptj'):
  tokens_num = get_tokens_num(user + ": " + utterance + "\n", model=model)
  item = {'user' : user, 'utterance' : utterance, 'tokens_num' : tokens_num}
  return item


def get_chat_log(chat_log_prompt, chat_log_dict, last_utt_only=False):
  chat_log = chat_log_prompt
  for i in chat_log_dict.keys():
    chat_log = chat_log + chat_log_dict[i]["user"] + ": " + chat_log_dict[i]["utterance"] + "\n"
  if last_utt_only:
    return chat_log_dict[len(chat_log_dict)-1]
  return chat_log

### Testing the models

Once again, uncomment the name of the model that you wish to test. The model must be loaded on the previous step.
Also, uncomment the name of the prompt that you would like to generate responses for.
If you wish to test your own prompt, create a txt file in prompts directory. Then create a json file of the same name in test_contexts directory. Refer to test files provided to understand the necessary format. Then, add the name of the prompt (as in txt and json file) to the prompts_list below.
The generation results for each prompt and each model will be written to models_output directory as a tsv file.

In [22]:
models = [
  # "gptj",
  # "gpt2_large",
  # "gpt_neo_2_7",
  # "opt_6_7b",
  # "bloomz",
  # "bloom",
  # "bloom_560m",
  # "opt_125m",
  #"bloom_7b",
]
prompts_list = [
  # "pizza",
  # "spacex",
  # "dreambot"
  ]

for model in models:
  for prompt_name in prompts_list:
    with open(f'test_contexts/{prompt_name}.json', 'r') as f:
      utts_test = json.load(f)

    with open(f'prompts/{prompt_name}.txt', 'r') as f:
      chat_log_prompt = f.read()
    chat_log_dict = {}
    prompts = []
    ans = []
    prompts.append(chat_log_prompt)
    chat_log = ""
    num_of_chat_items = 0
    n = 0
    for pack, dialogue in tqdm_notebook(utts_test.items()):
      chat = ''
      for utt in dialogue:
        #n = len(chat_log_dict) #for full context
        incoming_msg = utt
        if incoming_msg == "stop":
          break

        current_idx = len(chat_log_dict)
        chat_log = chat_log_prompt
        for key, value in chat_log_dict.items():
          if key > len(chat_log_dict) - n:
            chat_log = chat_log + chat_log_dict[key]["user"] + ": " + chat_log_dict[key]["utterance"] +'\n'
        chat_log_dict[current_idx+1] = create_chat_log_item("Human", incoming_msg, model=model)
        answer = ask(incoming_msg, chat_log, model=model) #!!!
        sub_str = "\nAI: "
        re = answer.split(sub_str)
        outgoing_msg=re[1]
        outgoing_msg=outgoing_msg.split("Human:")[0]
        chat_log_dict[current_idx+2] = create_chat_log_item("AI", outgoing_msg, model=model)
        chat += f"Human: {get_chat_log(chat_log_prompt, chat_log_dict, last_utt_only=True)['utterance']}\n"
        chat += f"AI: {outgoing_msg}\n\n"
      ans.append((prompt_name, pack, chat.replace('"', '')))
    df = pd.DataFrame(ans, columns =['prompt', 'utterances group', 'result'])
    df.to_csv(f'models_outputs/{prompt_name}/{model}_{prompt_name}.tsv', sep='\t', index=False)


/tmp/ipykernel_1232975/1893385311.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pack, dialogue in tqdm_notebook(utts_test.items()):


A Jupyter Widget